In [9]:
from simple_cysh import *

In [11]:
# get assessments
assmt_df = get_cysh_df('Assesment__c', ['Id', 'Type__c', 'Date_Administered__c',
                                        'X0_to_300_Scaled_Score__c', 'Student__c'])

# get assessment type
assmt_types = get_cysh_df('Picklist_Value__c', ['Id', 'Name'])
assmt_types = assmt_types.rename(columns={'Id': 'Type__c', 'Name':'Assessment Type'})

assmt_df = assmt_df.merge(assmt_types, how='left', on='Type__c')

assmt_df = assmt_df.loc[assmt_df['Assessment Type'].str.contains('NWEA')]

In [ ]:
## Next Steps
# pivot - one student per row, multiple tests as columns
# calculate fall to spring targets
# determine student growth
# associate students with ACM